In [1]:
%pip install fasttext

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-3.0.1-py3-none-any.whl.metadata (10.0 kB)
Using cached pybind11-3.0.1-py3-none-any.whl (293 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp313-cp313-macosx_14_0_universal2.whl size=664122 sha256=7b663cf62c12cbda8f730c36f81ba6f1fefed896dca6ba5882f9e793e8d770cd
  Stored in directory: /Users/chakrabortyb/Library/Caches/pip/wheels/59/06/3f/c95dbba0df6e58ba6ed18220c1aea1e96042802139df3674ef
Successfully built fasttext
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [fasttext]

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
# load training data
import urllib.request
import zipfile
import os

# datasets_dir = "datasets"
datasets_dir = "."
# os.makedirs(datasets_dir, exist_ok=True)

# download text8
url = "http://mattmahoney.net/dc/text8.zip"
zip_path = os.path.join(datasets_dir, "text8.zip")
text8_path = os.path.join(datasets_dir, "text8")

if not os.path.exists(text8_path):
    urllib.request.urlretrieve(url, zip_path)

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(datasets_dir)

with open(text8_path, 'r') as f:
    text8_data = f.read()

# create a smaller subset (200k characters)
text8_200k_path = os.path.join(datasets_dir, "text8_200k.txt")
text8_200k_data = text8_data[:200000].split()

if not os.path.exists(text8_200k_path):
    with open(text8_200k_path, 'w') as f:
        for token in text8_200k_data:
            f.write(token + "\n")

In [10]:
import fasttext
import os

model = fasttext.train_unsupervised(
    'text8_200k.txt',    # Input file
    model='skipgram',       
    dim=50,                 # Embedding dimension
    epoch=10,               # Number of training iterations
    minCount=1,             # Minimum word frequency
    minn=3,                 # Minimum character n-gram length
    maxn=6                  # Maximum character n-gram length
)

embeddings_path = os.path.join("..", "embeddings", "fasttext_embeddings.bin")
# model.save_model('fasttext_embeddings.bin')
model.save_model(embeddings_path)

Read 0M words
Number of words:  6064
Number of labels: 0
Progress: 100.0% words/sec/thread:  424069 lr:  0.000000 avg.loss:  4.121156 ETA:   0h 0m 0s


In [9]:
def get_word_embeddings(model):
    king_vector = model.get_word_vector('king')
    print(f"Vector for 'king': {king_vector[:5]}...")
    print(f"Vector shape: {king_vector.shape}")
    
    kingdom_vector = model.get_word_vector('kingdom')
    print(f"Vector for 'kingdom' (OOV): {kingdom_vector[:5]}...")
    
    return king_vector, kingdom_vector

king_vec, kingdom_vec = get_word_embeddings(model)

Vector for 'king': [ 0.03437728 -0.03446748  0.02068428  0.00712859 -0.00167755]...
Vector shape: (50,)
Vector for 'kingdom' (OOV): [ 0.01092913 -0.00803799  0.00401465  0.00038015  0.0015746 ]...
